# <span style="color:#E14F20"> Quantitative Comparison of BGFR algorithms // *For MRI QSM* </span> </br>

In [1]:
#So, we start the engine and then define SEPIA with all of its toolboxes:
import PyNomad as nomad
import matlab.engine
import os
eng = matlab.engine.start_matlab()

In [61]:
from python_to_matlab_wrap import python_to_sepia_bfr_wrap

In [33]:
# Wrapper functions 
sepia_path = "D:/Poly_MSc_Code/libraries_and_toolboxes/sepia"
xtra_tb_path = "D:/Poly_MSc_Code/libraries_and_toolboxes/toolboxes"

eng.addpath(sepia_path)
bfr_wrappers = eng.genpath("D:/Poly_MSc_Code/libraries_and_toolboxes/sepia/wrapper")
eng.addpath(bfr_wrappers, nargout=0)

## <span style="color:#285C2B"> LBV Optimization </span> </br>
- Input filename
- Header file
- Ouput basename
- mask filename

*LBV parameters* 
- Tolerance
- Depth
- Peel

*General algorithm parameters*
We can also add extra things like: </br>
- isBet 
- isInvert 
- refineMask 

*BGFR Algorithm paramters*
- refine_Method 
- refine_order 
- erode_radius 
- erode_before_radius 

But this needs to get packed into parameters, shown in code:

In [12]:
# So, lets try to replicate results from the MATLAB GUI
# Open the config file and copy the paths easily
input_fm_path = "E:\msc_data\sc_qsm\data\cropped/piece-wise/simulation/TE_1_weird_40/romeo_tests/test3_right_masked/autosepia"
header_path = "E:\msc_data\sc_qsm\data\cropped\piece-wise\simulation\TE_1_weird_40\header_qsm_tsting_hcrop2.mat"

In [15]:
# *General algorithm parameters
isBet = 0
isInvert = 0
refineMask = 0

# BGFR Algorithm paramters
refine_Method = None 
refine_order = 0
erode_radius = 0
erode_before_radius = 0

In [14]:
# LBV Parameters:
tolerance = 0.0001
depth = 5
peel = 2


In [59]:
eng.python_wrapper(in1, in2, in3, in4 , output_basename, mask_filename, bfr_params, nargout = 0)

### To test MATLAB engine install

In [ ]:
#Testing matlab 
# Start MATLAB
eng = matlab.engine.start_matlab()

# Test with a simple MATLAB command, like square root of 16
result = eng.sqrt(16.0)
print(f"Square root of 16 (from MATLAB): {result}")

# Close the MATLAB engine session
eng.quit()
